In [1]:
import itertools
from collections import defaultdict
from tqdm import tqdm
import _pickle as cPickle
import os

que_split_path_dict = {

#     "minval":  "../../datasets/VQA/back-translate/org2_bt_v2_OpenEnded_mscoco_minval2014_questions.pkl",
    "val":  "../../datasets/VQA/back-translate/org2_bt_v2_OpenEnded_mscoco_val2014_questions.pkl",
    "train": "../../datasets/VQA/back-translate/org2_bt_v2_OpenEnded_mscoco_train2014_questions.pkl",
}

answer_paths = [
#     "../../datasets/VQA/back-translate/org_bt_minval_target.pkl",
    "../../datasets/VQA/back-translate/org2_bt_val_target.pkl",
    "../../datasets/VQA/back-translate/org2_bt_train_target.pkl",
    ]


ans2label_path = os.path.join("../../datasets/VQA/", "cache", "trainval_ans2label.pkl")
label2ans_path = os.path.join("../../datasets/VQA/", "cache", "trainval_label2ans.pkl")

ans2label = cPickle.load(open(ans2label_path, "rb"))
label2ans = cPickle.load(open(label2ans_path, "rb"))
answer_dict = {}
# use later
answers_data = []

for path in (answer_paths):
    answers = cPickle.load(open(path, "rb"))
    answers_data.append(answers)
    answers = list(itertools.chain.from_iterable(answers))
    for ans in answers:
        answer_dict[ans["question_id"]] = ans["labels"]

print("Read Answers")

negs_path = "../../datasets/VQA/back-translate/train_val_question_negs.pkl"
negs_data = cPickle.load(open(negs_path, "rb"))
negs_dict = {}
for qid, sim_scores, sim_qids in zip(negs_data["qids"], negs_data["sim_scores"], negs_data["sim_qids"]):
    negs_dict[qid] = (sim_scores, sim_qids)

print("Read Matrix")

# create dicts
image_dict = defaultdict(list)
questions_rephrasings = defaultdict(list)
question_dict = {}

# use later
questions_data = []
qids_dict = {}
for split, que_path in que_split_path_dict.items():
    data = cPickle.load(open(que_path, "rb"))
    questions_data.append(data)
    questions_list = data["questions"]
    _dict = {}
    
    # add "rephrasing_of" key
    for _questions in questions_list:
        # only keep the min-qid in same-image ids
        min_qid = min([x['question_id'] for x in _questions])
        assert len(set([x['image_id'] for x in _questions])) == 1
        image_dict[_questions[0]["image_id"]].append(min_qid)
        for _que in _questions:
            question_dict[_que["question_id"]] = _que["question"]
            _dict[_que["question_id"]] = None
    qids_dict[split] = _dict

print("Read Questions")



Read Answers
Read Matrix
Read Questions


In [2]:
save_negs_dict_train = {}
save_negs_dict_val = {}
save_negs_dict_trainval = {}

def filter_negatives(sample):
    # filter same-image questions
    save_dict = {}
    same_image_ids = image_dict[sample["image_id"]]
    fil_same_image_ids = []
    ref_answers = answer_dict[sample["question_id"]]
    for qid in same_image_ids:
        if qid == sample["question_id"]:
            continue
        cand_answers = answer_dict[qid]
        if len(set(ref_answers).intersection(set(cand_answers))) == 0:
            fil_same_image_ids.append(qid)
    save_dict["same_image_questions_neg"] = fil_same_image_ids

    # filter top-k questions
    if sample["question_id"] not in negs_dict:
        return True

    top_k_sim_scores, top_k_questions = negs_dict[sample["question_id"]]
    fil_top_k_questions = []
    fil_top_k_questions_train = []
    fil_top_k_questions_val = []
    
    for qid in top_k_questions:
        cand_answers = answer_dict[qid]
        if len(set(ref_answers).intersection(set(cand_answers))) == 0:
            fil_top_k_questions.append(qid)
            if qid in qids_dict["train"]:
                fil_top_k_questions_train.append(qid)
            elif qid in qids_dict["val"]:
                fil_top_k_questions_val.append(qid)
            else:
                import pdb
                pdb.set_trace()
    
    save_dict["top_k_questions_neg"] = fil_top_k_questions
    save_dict["top_k_questions_neg_train"] = fil_top_k_questions_train
    save_dict["top_k_questions_neg_val"] = fil_top_k_questions_val

    if sample["question_id"] in qids_dict["train"]:
        del save_dict["top_k_questions_neg_val"]
        save_negs_dict_trainval[sample["question_id"]] = save_dict
        save_negs_dict_train[sample["question_id"]] = save_dict
    elif sample["question_id"] in qids_dict["val"]:
        del save_dict["top_k_questions_neg_train"]
        save_negs_dict_trainval[sample["question_id"]] = save_dict
        save_negs_dict_val[sample["question_id"]] = save_dict
    else:
        import pdb
        pdb.set_trace() 
    return False

In [3]:
for que_data, ans_data, que_path, ans_path in zip(questions_data, answers_data, que_split_path_dict.values(), answer_paths):
    data = que_data
    answers = ans_data

    questions_list = data["questions"]

    # add "rephrasing_of" key
    for _questions in questions_list:
        rep_id = min([s['question_id'] for s in _questions])
        for _que in _questions:
            _que["rephrasing_of"] = rep_id

    assert len(questions_list) == len(answers)

    # remove questions w/o negatives
    for idx in tqdm(range(len(questions_list)), total=len(questions_list)):
        _updated_ques = []
        _updated_answers = []
        _questions = questions_list[idx]
        _answers = answers[idx]

        for _que, _ans in zip(_questions, _answers):
            delete = filter_negatives(_que)
            if not delete:
                _updated_ques.append(_que)
                _updated_answers.append(_ans)
        questions_list[idx] = _updated_ques
        answers[idx] = _updated_answers

    cPickle.dump(data, open(que_path, "wb"), protocol=2)
    cPickle.dump(answers, open(ans_path, "wb"), protocol=2)
    print(f"Dumped: {que_path}")
    print(f"Dumped: {ans_path}")
    


100%|██████████| 214354/214354 [11:12<00:00, 318.77it/s]


Dumped: ../../datasets/VQA/back-translate/org2_bt_v2_OpenEnded_mscoco_val2014_questions.pkl
Dumped: ../../datasets/VQA/back-translate/org2_bt_val_target.pkl


100%|██████████| 443757/443757 [23:57<00:00, 308.77it/s]  


Dumped: ../../datasets/VQA/back-translate/org2_bt_v2_OpenEnded_mscoco_train2014_questions.pkl
Dumped: ../../datasets/VQA/back-translate/org2_bt_train_target.pkl


In [41]:
save_negs_paths = [
"../../datasets/VQA/back-translate/fil_val_question_negs.pkl",
"../../datasets/VQA/back-translate/fil_train_question_negs.pkl",
"../../datasets/VQA/back-translate/fil_trainval_question_negs.pkl",
]

# _dicts = [save_negs_dict_val,
# save_negs_dict_train,
# save_negs_dict_trainval]

# for path, split, _d in zip(save_negs_paths, ["val", "train", "trainval"], _dicts):
#     cPickle.dump(_d, open(path, "wb"), protocol=2)
#     print(f"Dumped: {path}")


In [ ]:
values = list(save_negs_dict_val.values())
lens = [len(x["top_k_questions_neg"]) for x in values]
max(lens)

In [62]:
import numpy as np

question_ids = list(save_negs_dict_val.keys())
question_values = list(save_negs_dict_val.values())
question_negs = np.zeros((len(question_ids), 300), dtype=np.int64) + -1
same_image_questions_neg = np.zeros((len(question_ids), 300), dtype=np.int64) + -1

for idx in tqdm(range(len(question_negs))):
    neg_ids = question_values[idx]["top_k_questions_neg_val"]
    same_image_ids = question_values[idx]["same_image_questions_neg"]
    question_negs[idx][:len(neg_ids)] = neg_ids
    try:
        same_image_questions_neg[idx][:len(same_image_ids)] = same_image_ids
    except:
        import pdb
        pdb.set_trace()




  0%|          | 0/1024112 [00:00<?, ?it/s]


  1%|          | 8482/1024112 [00:00<00:11, 84817.05it/s]


  2%|▏         | 16613/1024112 [00:00<00:12, 83733.41it/s]


  2%|▏         | 24727/1024112 [00:00<00:12, 82938.11it/s]


  3%|▎         | 32870/1024112 [00:00<00:12, 82479.69it/s]


  4%|▍         | 39670/1024112 [00:00<00:12, 77525.05it/s]


  5%|▍         | 48083/1024112 [00:00<00:12, 76600.22it/s]


  6%|▌         | 56430/1024112 [00:00<00:12, 78538.32it/s]


  6%|▋         | 64745/1024112 [00:00<00:12, 79866.29it/s]


  7%|▋         | 73007/1024112 [00:00<00:11, 80671.61it/s]


  8%|▊         | 81579/1024112 [00:01<00:11, 82119.89it/s]


  9%|▉         | 89822/1024112 [00:01<00:11, 82212.52it/s]


 10%|▉         | 97894/1024112 [00:01<00:11, 79859.58it/s]


 10%|█         | 105790/1024112 [00:01<00:11, 77552.05it/s]


 11%|█         | 113826/1024112 [00:01<00:11, 78373.42it/s]


 12%|█▏        | 121632/1024112 [00:01<00:11, 77709.62it/s]


 13%|█▎        | 129512/1024112 [0

 94%|█████████▍| 963028/1024112 [00:11<00:00, 76911.60it/s]


 95%|█████████▍| 970942/1024112 [00:11<00:00, 77565.06it/s]


 96%|█████████▌| 978975/1024112 [00:12<00:00, 78374.10it/s]


 96%|█████████▋| 986822/1024112 [00:12<00:00, 77439.37it/s]


 97%|█████████▋| 994575/1024112 [00:12<00:00, 77110.12it/s]


 98%|█████████▊| 1002293/1024112 [00:12<00:00, 75889.51it/s]


 99%|█████████▊| 1009891/1024112 [00:12<00:00, 75636.01it/s]


 99%|█████████▉| 1017602/1024112 [00:12<00:00, 76069.05it/s]


100%|██████████| 1024112/1024112 [00:12<00:00, 80707.23it/s]

In [64]:
cPickle.dump({"qids": question_ids, 
                    "question_negs": question_negs, 
                    "same_image_questions_neg": same_image_questions_neg}, 
             open(save_negs_paths[0], "wb"))


In [65]:
import numpy as np

question_ids = list(save_negs_dict_train.keys())
question_values = list(save_negs_dict_train.values())
question_negs = np.zeros((len(question_ids), 300), dtype=np.int64) + -1
same_image_questions_neg = np.zeros((len(question_ids), 300), dtype=np.int64) + -1

for idx in tqdm(range(len(question_negs))):
    neg_ids = question_values[idx]["top_k_questions_neg_train"]
    same_image_ids = question_values[idx]["same_image_questions_neg"]
    question_negs[idx][:len(neg_ids)] = neg_ids
    try:
        same_image_questions_neg[idx][:len(same_image_ids)] = same_image_ids
    except:
        import pdb
        pdb.set_trace()




  0%|          | 0/2119034 [00:00<?, ?it/s]


  0%|          | 5078/2119034 [00:00<00:41, 50774.96it/s]


  1%|          | 10845/2119034 [00:00<00:40, 52660.59it/s]


  1%|          | 16548/2119034 [00:00<00:39, 53898.67it/s]


  1%|          | 22290/2119034 [00:00<00:38, 54907.99it/s]


  1%|▏         | 27722/2119034 [00:00<00:38, 54728.66it/s]


  2%|▏         | 32554/2119034 [00:00<00:39, 52632.73it/s]


  2%|▏         | 38330/2119034 [00:00<00:38, 54071.90it/s]


  2%|▏         | 44184/2119034 [00:00<00:37, 55338.11it/s]


  2%|▏         | 49935/2119034 [00:00<00:36, 55971.77it/s]


  3%|▎         | 55730/2119034 [00:01<00:36, 56550.21it/s]


  3%|▎         | 61308/2119034 [00:01<00:36, 56315.27it/s]


  3%|▎         | 66949/2119034 [00:01<00:36, 56009.28it/s]


  3%|▎         | 72708/2119034 [00:01<00:36, 56470.68it/s]


  4%|▎         | 78318/2119034 [00:01<00:36, 56357.64it/s]


  4%|▍         | 83925/2119034 [00:01<00:36, 55926.57it/s]


  4%|▍         | 89498/2119034 [00:01

 30%|██▉       | 628225/2119034 [00:11<00:26, 55510.65it/s]


 30%|██▉       | 633783/2119034 [00:11<00:26, 55066.58it/s]


 30%|███       | 639296/2119034 [00:12<00:26, 54963.17it/s]


 30%|███       | 644957/2119034 [00:12<00:26, 55446.38it/s]


 31%|███       | 650506/2119034 [00:12<00:26, 54879.52it/s]


 31%|███       | 656228/2119034 [00:12<00:26, 55561.06it/s]


 31%|███       | 661903/2119034 [00:12<00:26, 55912.26it/s]


 32%|███▏      | 667728/2119034 [00:12<00:25, 56593.42it/s]


 32%|███▏      | 673393/2119034 [00:12<00:26, 55005.19it/s]


 32%|███▏      | 679048/2119034 [00:12<00:25, 55456.64it/s]


 32%|███▏      | 684725/2119034 [00:12<00:25, 55842.43it/s]


 33%|███▎      | 690317/2119034 [00:12<00:28, 50823.97it/s]


 33%|███▎      | 695652/2119034 [00:13<00:27, 51553.08it/s]


 33%|███▎      | 701359/2119034 [00:13<00:26, 53091.24it/s]


 33%|███▎      | 706779/2119034 [00:13<00:26, 53355.97it/s]


 34%|███▎      | 712442/2119034 [00:13<00:25, 54293.95it/s]


 34%|███

 59%|█████▊    | 1244616/2119034 [00:23<00:16, 51835.20it/s]


 59%|█████▉    | 1249817/2119034 [00:23<00:16, 51484.77it/s]


 59%|█████▉    | 1255463/2119034 [00:23<00:16, 52880.41it/s]


 59%|█████▉    | 1260770/2119034 [00:23<00:16, 52664.41it/s]


 60%|█████▉    | 1266254/2119034 [00:24<00:16, 53298.65it/s]


 60%|██████    | 1271595/2119034 [00:24<00:16, 52770.15it/s]


 60%|██████    | 1276881/2119034 [00:24<00:16, 52559.26it/s]


 61%|██████    | 1282144/2119034 [00:24<00:16, 51098.69it/s]


 61%|██████    | 1287281/2119034 [00:24<00:16, 51179.18it/s]


 61%|██████    | 1292409/2119034 [00:24<00:16, 50811.51it/s]


 61%|██████    | 1297498/2119034 [00:24<00:17, 48209.13it/s]


 61%|██████▏   | 1302836/2119034 [00:24<00:16, 49651.74it/s]


 62%|██████▏   | 1307997/2119034 [00:24<00:16, 50222.01it/s]


 62%|██████▏   | 1313219/2119034 [00:24<00:15, 50803.73it/s]


 62%|██████▏   | 1318642/2119034 [00:25<00:15, 51783.38it/s]


 62%|██████▏   | 1324136/2119034 [00:25<00:15, 52690.47

 87%|████████▋ | 1837291/2119034 [00:35<00:05, 52073.06it/s]


 87%|████████▋ | 1842511/2119034 [00:35<00:05, 51073.81it/s]


 87%|████████▋ | 1847713/2119034 [00:35<00:05, 51349.81it/s]


 87%|████████▋ | 1853034/2119034 [00:35<00:05, 51891.80it/s]


 88%|████████▊ | 1858232/2119034 [00:35<00:05, 51127.60it/s]


 88%|████████▊ | 1863481/2119034 [00:35<00:04, 51528.82it/s]


 88%|████████▊ | 1868641/2119034 [00:36<00:05, 46482.82it/s]


 88%|████████▊ | 1873566/2119034 [00:36<00:05, 47279.16it/s]


 89%|████████▊ | 1878367/2119034 [00:36<00:05, 45353.05it/s]


 89%|████████▉ | 1883698/2119034 [00:36<00:04, 47478.48it/s]


 89%|████████▉ | 1888859/2119034 [00:36<00:04, 48645.44it/s]


 89%|████████▉ | 1894141/2119034 [00:36<00:04, 49825.58it/s]


 90%|████████▉ | 1899172/2119034 [00:36<00:04, 49608.41it/s]


 90%|████████▉ | 1904167/2119034 [00:36<00:04, 48837.02it/s]


 90%|█████████ | 1909425/2119034 [00:36<00:04, 49899.36it/s]


 90%|█████████ | 1914440/2119034 [00:36<00:04, 47447.51

In [66]:
cPickle.dump({"qids": question_ids, 
                    "question_negs": question_negs, 
                    "same_image_questions_neg": same_image_questions_neg}, 
             open(save_negs_paths[1], "wb"))
print(save_negs_paths[1])

OverflowError: cannot serialize a bytes object larger than 4 GiB

In [ ]:
import numpy as np

question_ids = list(save_negs_dict_trainval.keys())
question_values = list(save_negs_dict_trainval.values())
question_negs = np.zeros((len(question_ids), 300), dtype=np.int64) + -1
same_image_questions_neg = np.zeros((len(question_ids), 300), dtype=np.int64) + -1

for idx in tqdm(range(len(question_negs))):
    neg_ids = question_values[idx]["top_k_questions_neg_trainval"]
    same_image_ids = question_values[idx]["same_image_questions_neg"]
    question_negs[idx][:len(neg_ids)] = neg_ids
    try:
        same_image_questions_neg[idx][:len(same_image_ids)] = same_image_ids
    except:
        import pdb
        pdb.set_trace()

In [ ]:
cPickle.dump({"qids": question_ids, 
                    "question_negs": question_negs, 
                    "same_image_questions_neg": same_image_questions_neg}, 
             open(save_negs_paths[2], "wb"))
print(save_negs_paths[2])

In [47]:
d = cPickle.load(open(save_negs_paths[2], "rb"))

In [61]:
question_negs[-2]

array([   41231005,   227224579,   247906000,   710156291, -1402375165,
        1628983298,  1628983296,   154496006,     6318004,    47813001,
         387048003,   602631171,   461281003,   202914001,   539617001,
         113026022,    92746007,   263552000,   263552001,   263552002,
         263552003, -1995043837, -1679023104,      634001,  -920793085,
        -188925949,    14307001,    72564002,   569301007,  -581186558,
         208095001, -1918048254,  1030046720,  1563226115,   467872002,
       -1519958013, -1147418624,  -571514880,   706717699,   728522755,
        1030046721, -1924250622,   161843002,  1997336579,   548384002,
        1997336578,   395080005,  1997336577,    49382003, -1147418622,
         726726659,   230503002,   205744002,   463451000,   791439362,
         850798592,   262893004,   850798594,    62761000,   484714053,
        2064413696,   515274002,  1841353731,   203151000,   351610003,
        -419241981,   325566467,   439974003,   791439360,   355